In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [51.0 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [770 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.co

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT YEAR(date), ROUND(AVG(price), 2)
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
"""
spark.sql(query).show()


+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2019|            300263.7|
|      2020|           298353.78|
|      2021|           301819.44|
+----------+--------------------+



In [9]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
"""
spark.sql(query).show()

+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [10]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built DESC
"""
spark.sql(query).show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [12]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.9202322959899902 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.5161924362182617 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('p_home_sales', mode='overwrite')

In [18]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('p_home_sales')

In [19]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS Average_Price
FROM p_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|Average_Price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.2769486904144287 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")
spark.sql("uncache table p_home_sales")

DataFrame[]

In [23]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("home_sales is still cached")
else:
  print("all clear")


all clear
